# OpenAI API 사용 Function Calling 
: https://www.promptingguide.ai/kr/applications/function_calling

In [18]:
import os
import openai
import json
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")
# print(API_KEY)

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

### 질의 함수 정의하기

In [19]:
def get_completion(messages, model="gpt-4o-mini", temperature=0, max_tokens=300, tools=None, tool_choice=None):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        tools=tools,
        tool_choice=tool_choice
    )
    return response.choices[0].message

In [22]:
import json

# 함수 정의
def get_current_weather(location):
    # 이 함수는 날씨 정보를 반환한다고 가정합니다.
    print('get_current_weather function is called!!')
    return {
        "location": location,
        "temperature": "25°C",
        "description": "구름조금 있고 바람붐"
    }

# Function Calling을 위한 함수 메타데이터 정의
functions = [
    {
        "name": "get_current_weather",
        "description": "지정된 위치의 현재 날씨를 가져옵니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "도시와 국가 예: 서울, 대한민국"
                }
            },
            "required": ["location"]
        }
    }
]

# 사용자가 원하는 위치의 날씨를 묻는 프롬프트
prompt = "서울의 날씨는 어떤가요?"
# prompt = "뉴욕의 날씨는 어떤가요?"
prompt = "파리의 날씨는 어떤가요?"

# ChatGPT API 호출
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    functions=functions,
    function_call="auto"  # Function Calling을 활성화하기 위한 설정
)

# function_call={"name": "get_current_weather"}라고 지정하면, 
# 모델은 반드시 get_current_weather 함수를 호출합니다. 그러나 인자 전달 오류
print(response)
print('-'*100)

# Function Call이 발생했는지 확인 후 직접 함수 호출
if response.choices[0].finish_reason == "function_call":
    function_call = response.choices[0].message.function_call

    if function_call.name == "get_current_weather":
        location = json.loads(function_call.arguments)["location"]
        weather_info = get_current_weather(location)

        # 사용자가 요청한 정보에 대한 응답 생성
        print(f"{location}의 현재 날씨는 {weather_info['temperature']}이고, {weather_info['description']}입니다.")
else:
    # Function Calling이 발생하지 않았을 때의 응답 처리
    print(response.choices[0].message['content'])


ChatCompletion(id='chatcmpl-AH4F64HbyqXNavk7vCYDVttQWF53I', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Paris, France"}', name='get_current_weather'), tool_calls=None))], created=1728631248, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_e2bde53e6e', usage=CompletionUsage(completion_tokens=17, prompt_tokens=76, total_tokens=93, prompt_tokens_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0}))
----------------------------------------------------------------------------------------------------
get_current_weather function is called!!
Paris, France의 현재 날씨는 25°C이고, 구름조금 있고 바람붐입니다.


### Dummy Function 정의

In [4]:
# 현재 날씨를 가져오는 dummy 함수를 정의
# 실제 검색 가능한 라이브러리 함수와 연동하여 사용한다
def get_current_weather(location, unit="섭씨"):
    """Get the current weather in a given location"""
    weather = {
        "location": location,
        "temperature": "30",
        "unit": unit,
    }
    return json.dumps(weather)

### 함수 정의

OpenAI 문서에서 시연된 바와 같이, 요청에 포함될 함수를 정의하는 간단한 예시가 있습니다.

descriptions는 매우 중요합니다. 이 설명은 LLM에 직접 전달되며, LLM은 descriptions을 바탕으로 함수를 사용할지, 그리고 어떻게 호출할지를 결정하게 됩니다.

In [5]:
# define a function as tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "주어진 위치의 현재 날씨를 가져오기",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "도시, 예. 서울",
                    },
                    "unit": {
                        "type": "string", 
                        "enum": ["섭씨", "화씨"]},
                },
                "required": ["location"],
            },
        },   
    }
]

In [6]:
# define a list of messages
messages = [
    {
        "role": "user",
        "content": "서울의 날씨는 어때?"
    }
]

In [7]:
response = get_completion(messages, tools=tools)
print(response)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Xd17UNvlvU1IkpSj7ENHrvle', function=Function(arguments='{"location":"서울","unit":"섭씨"}', name='get_current_weather'), type='function')])


In [8]:
response.tool_calls[0].function.arguments

'{"location":"서울","unit":"섭씨"}'

이제 인수를 캡처할 수 있습니다:

In [9]:
args = json.loads(response.tool_calls[0].function.arguments)

In [10]:
get_current_weather(**args)

'{"location": "\\uc11c\\uc6b8", "temperature": "30", "unit": "\\uc12d\\uc528"}'

### Function Calling 동작 제어하기

LLM 기반 대화 에이전트의 맥락에서 이 function_calling 기능을 설계하는 데 관심이 있다고 가정해 보겠습니다. 그런 다음 솔루션은 어떤 함수를 호출해야 하는지 또는 호출해야 하는지 알아야 합니다. 인사말 메시지의 간단한 예를 살펴보겠습니다.

In [11]:
messages = [
    {
        "role": "user",
        "content": "안녕 반가워?",
    }
]

In [12]:
get_completion(messages, tools=tools)

ChatCompletionMessage(content='안녕하세요! 반갑습니다. 어떻게 도와드릴까요?', refusal=None, role='assistant', function_call=None, tool_calls=None)

함수 호출의 동작을 제어하기 위해 원하는 동작을 지정할 수 있습니다. 기본적으로, 모델은 함수를 호출할지 여부와 어떤 함수를 호출할지를 스스로 결정합니다. 이는 기본 설정인 tool_choice: "auto"를 설정함으로써 이루어집니다.

In [13]:
get_completion(messages, tools=tools, tool_choice="auto")

ChatCompletionMessage(content='안녕하세요! 반갑습니다. 어떻게 도와드릴까요?', refusal=None, role='assistant', function_call=None, tool_calls=None)

tool_choice: "none"으로 설정하면 모델이 제공된 함수 중 어떤 것도 사용하지 않도록 강제합니다.

In [14]:
get_completion(messages, tools=tools, tool_choice="none")

ChatCompletionMessage(content='안녕하세요! 반갑습니다. 어떻게 도와드릴까요?', refusal=None, role='assistant', function_call=None, tool_calls=None)

In [15]:
messages = [
    {
        "role": "user",
        "content": "부산의 날씨는 어때?",
    }
]
get_completion(messages, tools=tools, tool_choice="none")

ChatCompletionMessage(content='부산의 현재 날씨를 확인해보겠습니다. 잠시만 기다려 주세요.', refusal=None, role='assistant', function_call=None, tool_calls=None)

또한, 애플리케이션에서 원하는 동작이 있다면 모델이 반드시 함수를 선택하도록 강제할 수도 있습니다.
예시:

In [16]:
messages = [
    {
        "role": "user",
        "content": "부산의 날씨는 어때?",
    }
]
get_completion(messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_current_weather"}})

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_gU6OyHnINyoeWq7iUrH0SrPX', function=Function(arguments='{"location":"부산","unit":"섭씨"}', name='get_current_weather'), type='function')])

OpenAI API는 하나의 턴에서 여러 함수를 호출할 수 있는 병렬 함수 호출도 지원합니다. 

In [17]:
messages = [
    {
        "role": "user",
        "content": "다음 며칠 동안 서울과 부산의 날씨는 어떤가요?",
    }
]
get_completion(messages, tools=tools)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_3ABJuZI02fyAsgIMem4cIpDb', function=Function(arguments='{"location": "서울", "unit": "섭씨"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_dryLjpJP8ufiQMGrwvcRNDkK', function=Function(arguments='{"location": "부산", "unit": "섭씨"}', name='get_current_weather'), type='function')])

You can see in the response above that the response contains information from the function calls for the two locations queried. 

### 모델 피드백을 위한 Function Calling Response 

또한, 함수 호출로 생성된 입력을 사용하여 API를 호출한 후 얻은 결과를 반환하는 에이전트를 개발하는 것에 관심이 있을 수 있습니다. 다음은 그 예시입니다:


In [18]:
messages = []
messages.append({"role": "user", "content": "대구의 날씨는 어때!"})
assistant_message = get_completion(messages, tools=tools, tool_choice="auto")
assistant_message = json.loads(assistant_message.model_dump_json())
assistant_message["content"] = str(assistant_message["tool_calls"][0]["function"])

# a temporary patch but this should be handled differently
# remove "function_call" from assistant message
del assistant_message["function_call"]

In [19]:
messages.append(assistant_message)

In [20]:
messages

[{'role': 'user', 'content': '대구의 날씨는 어때!'},
 {'content': '{\'arguments\': \'{"location":"대구","unit":"섭씨"}\', \'name\': \'get_current_weather\'}',
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_ID0jlasTaOa8wHnj32zCvGMw',
    'function': {'arguments': '{"location":"대구","unit":"섭씨"}',
     'name': 'get_current_weather'},
    'type': 'function'}]}]

그런 다음 get_current_weather 함수의 결과를 추가하고 이를 도구 역할을 사용하여 모델에 다시 전달합니다

In [21]:
# get the weather information to pass back to the model
weather = get_current_weather(messages[1]["tool_calls"][0]["function"]["arguments"])

messages.append({"role": "tool",
                 "tool_call_id": assistant_message["tool_calls"][0]["id"],
                 "name": assistant_message["tool_calls"][0]["function"]["name"],
                 "content": weather})

In [22]:
messages

[{'role': 'user', 'content': '대구의 날씨는 어때!'},
 {'content': '{\'arguments\': \'{"location":"대구","unit":"섭씨"}\', \'name\': \'get_current_weather\'}',
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_ID0jlasTaOa8wHnj32zCvGMw',
    'function': {'arguments': '{"location":"대구","unit":"섭씨"}',
     'name': 'get_current_weather'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_ID0jlasTaOa8wHnj32zCvGMw',
  'name': 'get_current_weather',
  'content': '{"location": "{\\"location\\":\\"\\ub300\\uad6c\\",\\"unit\\":\\"\\uc12d\\uc528\\"}", "temperature": "30", "unit": "\\uc12d\\uc528"}'}]

In [23]:
final_response = get_completion(messages, tools=tools)

In [24]:
final_response

ChatCompletionMessage(content='대구의 현재 기온은 30도 섭씨입니다. 날씨가 꽤 덥네요!', refusal=None, role='assistant', function_call=None, tool_calls=None)